# ImmunOmics — Multi-Omics Integration Demo

> **Author:** Patrick Grady  
> **Repository:** [github.com/pgrady1322/immunomics](https://github.com/pgrady1322/immunomics)  
> **Version:** 0.1.0

This notebook demonstrates the ImmunOmics pipeline:

1. **Data preprocessing** — scRNA-seq (QC → normalize → HVG → PCA) and scATAC-seq (QC → TF-IDF → LSI)
2. **Differential analysis** — DE genes and DA peaks per cell type
3. **TF activity inference** — Expression-based transcription factor activity
4. **Peak-to-gene linkage** — Correlation-based enhancer-gene mapping
5. **Visualization** — TF heatmaps, peak-gene link plots, integration comparisons

We use **synthetic multi-omics data** so this notebook runs without downloading any datasets.

In [ ]:
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print("ImmunOmics demo — packages loaded successfully")

## 1. Create Synthetic Multi-Omics Data

We simulate matched scRNA-seq and scATAC-seq data from 500 immune cells across 4 cell types, with realistic structure including mitochondrial genes, known TF genes, and properly formatted ATAC peak names.

In [ ]:
np.random.seed(42)
n_cells = 500
cell_types = np.random.choice(["T cell", "B cell", "Monocyte", "NK cell"], n_cells, p=[0.3, 0.25, 0.25, 0.2])

# --- RNA data ---
immune_tfs = ["TBX21", "GATA3", "PAX5", "SPI1", "FOXP3", "IRF4", "EOMES", "RORC", "BCL6", "CEBPA"]
mt_genes = [f"MT-Gene_{i}" for i in range(5)]
other_genes = [f"Gene_{i}" for i in range(485)]
gene_names = immune_tfs + mt_genes + other_genes
n_genes = len(gene_names)

rna_X = np.random.rand(n_cells, n_genes).astype(np.float32) * 5
# Add cell-type-specific signal to TFs
for i, ct in enumerate(cell_types):
    if ct == "T cell":
        rna_X[i, 0:3] *= 3  # TBX21, GATA3, PAX5 upregulated in T cells
    elif ct == "B cell":
        rna_X[i, 3:5] *= 3  # SPI1, FOXP3
    elif ct == "Monocyte":
        rna_X[i, 5:8] *= 3  # IRF4, EOMES, RORC
    elif ct == "NK cell":
        rna_X[i, 8:10] *= 3  # BCL6, CEBPA

adata_rna = ad.AnnData(X=csr_matrix(rna_X))
adata_rna.var_names = pd.Index(gene_names)
adata_rna.obs_names = [f"cell_{i}" for i in range(n_cells)]
adata_rna.obs["cell_type"] = cell_types
adata_rna.layers["counts"] = adata_rna.X.copy()

# --- ATAC data ---
n_peaks = 1000
peak_names = [f"chr{(i % 22) + 1}:{i * 500}-{i * 500 + 300}" for i in range(n_peaks)]
atac_X = (np.random.rand(n_cells, n_peaks) > 0.85).astype(np.float32)

adata_atac = ad.AnnData(X=csr_matrix(atac_X))
adata_atac.var_names = pd.Index(peak_names)
adata_atac.obs_names = [f"cell_{i}" for i in range(n_cells)]
adata_atac.obs["cell_type"] = cell_types
adata_atac.layers["counts"] = adata_atac.X.copy()

print(f"RNA:  {adata_rna.n_obs} cells × {adata_rna.n_vars} genes  (incl. {len(immune_tfs)} TFs, {len(mt_genes)} MT genes)")
print(f"ATAC: {adata_atac.n_obs} cells × {adata_atac.n_vars} peaks")
print(f"Cell types: {dict(zip(*np.unique(cell_types, return_counts=True)))}")

## 2. RNA Preprocessing

Standard scRNA-seq pipeline: QC filtering → library normalization → log-transform → HVG selection (Seurat v3) → scaling → PCA.

In [ ]:
from immunomics.data.preprocess_rna import preprocess_rna

adata_rna_pp = preprocess_rna(
    adata_rna,
    min_genes=10,       # Lower thresholds for synthetic data
    min_cells=1,
    max_pct_mito=50.0,
    n_top_genes=200,
    n_pcs=30,
)

print(f"After preprocessing: {adata_rna_pp.n_obs} cells, {adata_rna_pp.n_vars} genes")
print(f"PCA: {adata_rna_pp.obsm['X_pca'].shape[1]} components")
print(f"HVGs: {adata_rna_pp.var['highly_variable'].sum()}")

## 3. ATAC Preprocessing

scATAC-seq pipeline: QC filtering → peak selection → TF-IDF normalization → LSI (Latent Semantic Indexing) via truncated SVD. The first LSI component is excluded (correlates with sequencing depth).

In [ ]:
from immunomics.data.preprocess_atac import preprocess_atac, parse_peak_coordinates

adata_atac_pp = preprocess_atac(
    adata_atac,
    min_peaks=1,
    min_cells=1,
    n_components=20,
)

# Parse peak coordinates
adata_atac_pp = parse_peak_coordinates(adata_atac_pp)

print(f"After preprocessing: {adata_atac_pp.n_obs} cells, {adata_atac_pp.n_vars} peaks")
print(f"LSI: {adata_atac_pp.obsm['X_lsi'].shape[1]} components")
print(f"Total explained variance: {adata_atac_pp.uns['lsi_variance_ratio'].sum():.3f}")
print(f"\nPeak coordinate columns: {list(adata_atac_pp.var[['chrom', 'start', 'end']].columns)}")
adata_atac_pp.var[['chrom', 'start', 'end']].head()

## 4. Differential Gene Expression

Find differentially expressed genes between cell types using the Wilcoxon rank-sum test (via scanpy).

In [ ]:
from immunomics.analysis.differential import differential_genes, differential_peaks

# DE genes
de_results = differential_genes(adata_rna_pp, groupby="cell_type", n_genes=10)
print(f"DE results: {len(de_results)} gene-group pairs")
print(f"Groups tested: {de_results['group'].unique().tolist()}")
print()

# Show top DE genes per group
for group in de_results['group'].unique():
    top = de_results[de_results['group'] == group].head(3)
    genes = top['names'].tolist()
    scores = top['scores'].round(2).tolist()
    print(f"  {group}: {list(zip(genes, scores))}")

## 5. TF Activity Inference

Infer transcription factor activity from expression data. ImmunOmics computes per-cell-type expression z-scores and rank-based activity scores for curated immune TFs.

In [ ]:
from immunomics.analysis.tf_activity import infer_tf_activity, IMMUNE_TFS

# Show curated immune TF categories
print("Curated Immune TF Categories:")
for category, tfs in IMMUNE_TFS.items():
    print(f"  {category}: {', '.join(tfs)}")

# Infer TF activity (use RNA for both since we don't have real ATAC motif data)
tf_activity = infer_tf_activity(
    adata_rna,   # Unprocessed RNA with TF genes
    adata_atac,  # Shared barcodes
    tf_list=["TBX21", "GATA3", "PAX5", "SPI1", "FOXP3", "IRF4", "EOMES", "RORC"],
)

print(f"\nTF activity results: {len(tf_activity)} rows")
print(f"Columns: {tf_activity.columns.tolist()}")
tf_activity.head(8)

## 6. Visualization — TF Activity Heatmap

Visualize transcription factor activity across immune cell types as a clustered heatmap.

In [ ]:
from immunomics.visualization.plots import plot_tf_heatmap

fig = plot_tf_heatmap(tf_activity, top_n=8, figsize=(10, 6))
plt.show()

## 7. Peak-to-Gene Linkage

Link ATAC peaks to target genes by correlation analysis. We provide synthetic gene annotations to avoid querying pybiomart.

In [ ]:
from immunomics.analysis.peak_gene_links import link_peaks_to_genes

# Create synthetic gene TSS annotations matching our peak locations
gene_annotation = pd.DataFrame({
    "gene_name": [f"Gene_{i}" for i in range(50)],
    "chrom": [f"chr{(i % 22) + 1}" for i in range(50)],
    "tss": [i * 500 + 150 for i in range(50)],  # Near our synthetic peaks
})

# Normalize RNA for correlation
adata_rna_norm = adata_rna.copy()
sc.pp.normalize_total(adata_rna_norm, target_sum=1e4)
sc.pp.log1p(adata_rna_norm)

links = link_peaks_to_genes(
    adata_rna_norm,
    adata_atac,
    max_distance=50000,
    min_correlation=0.05,
    pvalue_threshold=0.5,  # Relaxed for synthetic data
    gene_annotation=gene_annotation,
)

print(f"Peak-gene links found: {len(links)}")
if len(links) > 0:
    print(f"\nTop 5 links:")
    print(links.head())

## 8. Visualization — Integration Method Comparison

Simulate benchmark results to demonstrate the comparison visualization.

In [ ]:
from immunomics.visualization.plots import plot_integration_comparison

# Simulated benchmark results (real benchmarks need scvi-tools, R, etc.)
benchmark_results = {
    "multivi": {
        "silhouette_score": 0.42,
        "adjusted_rand_index": 0.55,
        "runtime_seconds": 180,
    },
    "mofa": {
        "silhouette_score": 0.38,
        "adjusted_rand_index": 0.48,
        "runtime_seconds": 45,
    },
    "wnn": {
        "silhouette_score": 0.45,
        "adjusted_rand_index": 0.52,
        "runtime_seconds": 30,
    },
}

fig = plot_integration_comparison(benchmark_results, figsize=(9, 5))
plt.show()

## 9. Configuration System

ImmunOmics uses YAML configuration files to parameterize the full pipeline — from preprocessing thresholds to integration model hyperparameters.

In [ ]:
from immunomics.utils.config import load_config
import json

config = load_config("../configs/integration.yaml")

# Show key config sections
for section in ["rna_preprocessing", "atac_preprocessing", "integration"]:
    print(f"\n── {section} ──")
    print(json.dumps(config[section], indent=2))

## Summary

This notebook demonstrated the core ImmunOmics pipeline components:

| Step | Module | Status |
|---|---|---|
| RNA preprocessing | `immunomics.data.preprocess_rna` | QC → normalize → HVG → PCA |
| ATAC preprocessing | `immunomics.data.preprocess_atac` | QC → TF-IDF → LSI |
| Differential expression | `immunomics.analysis.differential` | Wilcoxon rank-sum per cell type |
| TF activity | `immunomics.analysis.tf_activity` | Expression z-scores + rank activity |
| Peak-gene links | `immunomics.analysis.peak_gene_links` | Pearson correlation + BH correction |
| Visualization | `immunomics.visualization.plots` | TF heatmap, benchmark comparison |
| Configuration | `immunomics.utils.config` | YAML-driven pipeline parameters |

**For real data integration** (MultiVI, WNN, MOFA+), install the optional backends:
```bash
pip install -e ".[ml]"  # scvi-tools for MultiVI
# or for WNN: conda install -c conda-forge rpy2 r-seurat r-signac
```